## Instalacja Sparka na platformie Colab

In [1]:
#Instalacja Sparka na colabie
!git clone https://github.com/djkormo/colab-examples.git

Cloning into 'colab-examples'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 103 (delta 30), reused 96 (delta 23), pack-reused 0
Receiving objects: 100% (103/103), 6.84 MiB | 16.96 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [2]:
!bash colab-examples/spark/install.bash

mkdir: cannot create directory ‘content’: File exists
Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk-headless is already the newest version (8u222-b10-1ubuntu1~18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
--2019-10-14 20:55:58--  http://apache.crihan.fr/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
Resolving apache.crihan.fr (apache.crihan.fr)... 195.221.21.36, 2001:660:7401:211::36
Connecting to apache.crihan.fr (apache.crihan.fr)|195.221.21.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230091034 (219M) [application/x-gzip]
Saving to: ‘spark-2.4.4-bin-hadoop2.7.tgz’

spark-2.4.4-bin-had 100%[===================>] 219.43M  27.0MB/s    in 8.8s    

2019-10-14 20:56:07 (24.8 MB/s) - ‘spark-2.4.4-bin-hadoop2.7.tgz’ saved [230091034/230091034]



In [0]:
# based on https://www.tutorialspoint.com/pyspark/pyspark_sparkcontext.htm

import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_ME'] = '/content/spark-2.4.4-bin-hadoop2.7'

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("First App") \
    .master("local[*]")\
    .getOrCreate()


In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

## Spark DataFrames

### Import

Dla przykładów omawianych poniżej będą wymagane dwie bliblioteki z pakietu  **pyspark.sql**:
- `types`
- `functions`

Do zdefiniowania schematów dla DataFrames potrzebujemy `pyspark.sql.types`. Biblioteka `pyspark.sql.functions` zawiera wszystkie funkcje specyficzne dla SQL i DataFrames w ** PySpark **.

In [0]:
from pyspark.sql.types import *  # Konieczna do tworzenia schematów
from pyspark.sql.functions import * # Importowanie funkcji PySpark

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

### Tworzenie obiektu DataFrames

#### Utwórz obiekt DataFrame z listy krotek

In [9]:
# Stwórz listę krotek
a_list = [('a', 1), ('b', 2), ('c', 3)]

# Stwórz Spark DataFrame bez podawania szczegółów schamtu
df_from_list_no_schema = sqlContext.createDataFrame(a_list)

# Wyświetl obiekt DataFrame
print (df_from_list_no_schema)

# Wyświetl listę obiektów typu Row
print (df_from_list_no_schema.collect())

# Wyświetl zawartość obiektu DataFrame
df_from_list_no_schema.show()

DataFrame[_1: string, _2: bigint]
[Row(_1='a', _2=1), Row(_1='b', _2=2), Row(_1='c', _2=3)]
+---+---+
| _1| _2|
+---+---+
|  a|  1|
|  b|  2|
|  c|  3|
+---+---+



#### Tworzenie obiektu DataFrame z listy krotek i określonym schematem

In [10]:
# Tym razem utwórz obiekt DataFrme wraz z określonym schematem danych
df_from_list_with_schema = \
sqlContext.createDataFrame(a_list, ['letters', 'numbers']) # ten prosty schemat zawiera tylko nazwy kolumn

# Wyświetl obiekt DataFrame
df_from_list_with_schema.show()

# Wyświetl schemat obiektu
df_from_list_with_schema.printSchema()

+-------+-------+
|letters|numbers|
+-------+-------+
|      a|      1|
|      b|      2|
|      c|      3|
+-------+-------+

root
 |-- letters: string (nullable = true)
 |-- numbers: long (nullable = true)



#### Making a Simple DataFrame from a Dictionary

In [11]:
# Stwórz strukturę słownika
a_dict = [{'letters': 'a', 'numbers': 1},
          {'letters': 'b', 'numbers': 2},
          {'letters': 'c', 'numbers': 3}]

# Stwórz obiekt Spark DataFrame używając słownika
df_from_dict = \
(sqlContext
 .createDataFrame(a_dict)) # Zwróć uwagę na ostrzeżenie

# Show the DataFrame
df_from_dict.show()

spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-------+-------+
|letters|numbers|
+-------+-------+
|      a|      1|
|      b|      2|
|      c|      3|
+-------+-------+



In [12]:
# Podobnie jak przykład wyżej, w tym, że bez ostrzeżenia
from pyspark.sql import Row

# Stwórz słownik
a_dict = [{'letters': 'a', 'numbers': 1},
          {'letters': 'b', 'numbers': 2},
          {'letters': 'c', 'numbers': 3}]

# Stwórz obiekt Spark DataFrame używając słownika
# Tutaj użyłem ** (kwargs keyword argument unpacking), aby przekazać słowniki do konstruktora obiektu Row.
df_from_dict =  sqlContext.createDataFrame(map(lambda x : Row(**x), a_dict)) 

# Wyświetl obiekt DataFrame
df_from_dict.show()

+-------+-------+
|letters|numbers|
+-------+-------+
|      a|      1|
|      b|      2|
|      c|      3|
+-------+-------+



In [0]:
# Wyświel zawartość argumentu zawierającego klucz-wartość

def print_kwargs(**kwargs):
    for i in kwargs:
        print(i, kwargs[i])

In [14]:
# wyświetl pierwszy wiersz zdefiniowanego słownika
print_kwargs(**a_dict[0])

letters a
numbers 1


In [15]:
from pyspark.sql import Row

# utwórz słownik
a_dict = [{'letters': 'a', 'numbers': 1},
          {'letters': 'b', 'numbers': 2},
          {'letters': 'c', 'numbers': 3}]

# Stwórz obiekt Spark DataFrame używając słownika
# Tutaj użyłem ** (kwargs keyword argument unpacking), aby przekazać słowniki do konstruktora Row.
df_from_dict =  sqlContext.createDataFrame(Row(**x) for x in a_dict) # z zastosowaniem list comprehension (listy składane)

# Wyświetl obiekt DataFrame
df_from_dict.show()


+-------+-------+
|letters|numbers|
+-------+-------+
|      a|      1|
|      b|      2|
|      c|      3|
+-------+-------+



#### Tworzenie obiektu DataFrame z zastosowaniem StructType Schema i obiektów RDD

In [16]:
# Zdefiniuj schemat
schema = StructType([
    StructField('letters', StringType(), True),
    StructField('numbers', IntegerType(), True)])

# Utwórz obiekty RDD z wcześniej zdefiniowanej listy
rdd = sc.parallelize(a_list)

# Utwórz obiekt DataFrame z wcześniej utworzonych komponentów
nice_df = sqlContext.createDataFrame(rdd, schema)

# Wyświetl obiekt DataFrame
nice_df.show()

+-------+-------+
|letters|numbers|
+-------+-------+
|      a|      1|
|      b|      2|
|      c|      3|
+-------+-------+



### Proste funkcje kontrolne

Mamy obiekt `nice_df`, oto kilka przydatnych funkcji służących do inspekcji obiektów DataFrame.

In [17]:
# `columns`: zwraca wszystkie nazwy kolumn jako listę
nice_df.columns

['letters', 'numbers']

In [18]:
# `dtypes`: pobierz typy danych dla wszystkich kolumn
nice_df.dtypes

[('letters', 'string'), ('numbers', 'int')]

In [19]:
# `printSchema()`: wyświetla schemat danych sprawdzanego obiektu DataFrame
nice_df.printSchema()

root
 |-- letters: string (nullable = true)
 |-- numbers: integer (nullable = true)



In [20]:
# `schema`: zwraca schemat danych jako obiekt `StructType`
nice_df.schema

StructType(List(StructField(letters,StringType,true),StructField(numbers,IntegerType,true)))

In [21]:
# `first()` zwraca pierwszy wiersz jako obiekt Row
# `head()` oraz `take()` zwracają `n` kolejnych obiektów Row - tylko z początku obiektu DataFrame
print(nice_df.first()) # bez argumentów
print(nice_df.head(2)) # może opcjonalnie podać wartość (domyślnie: 1)
                       # z zakresu liczba całkowitych n > 1, zwraca listę
print(nice_df.take(2)) # oczekiwana wartość całkowita n >=1, zwraca listę

Row(letters='a', numbers=1)
[Row(letters='a', numbers=1), Row(letters='b', numbers=2)]
[Row(letters='a', numbers=1), Row(letters='b', numbers=2)]


In [22]:
# `count()`: zwraca liczbę wszystkich wierszy w DF
nice_df.count()

3

In [23]:
# `describe()`: wyświetla statystykę kolumn zawierających wartości numeryczne
nice_df.describe().show() # opcjonalnie można dodać jako argument listę interesująch nas kolumn

+-------+-------+-------+
|summary|letters|numbers|
+-------+-------+-------+
|  count|      3|      3|
|   mean|   null|    2.0|
| stddev|   null|    1.0|
|    min|      a|      1|
|    max|      c|      3|
+-------+-------+-------+



In [24]:
# `describe()`: wyświetla statystykę kolumn zawierających wartości numeryczne
nice_df.describe(['numbers']).show() # opcjonalnie można dodać jako argument listę interesująch nas kolumn

+-------+-------+
|summary|numbers|
+-------+-------+
|  count|      3|
|   mean|    2.0|
| stddev|    1.0|
|    min|      1|
|    max|      3|
+-------+-------+



In [26]:
# the `explain()`: funkcaj wyświetla co dzieje się "pod maską" w procesie ewaliacji transormacji 
nice_df.explain()

== Physical Plan ==
Scan ExistingRDD[letters#55,numbers#56]


### Względnie proste funkcje pozwalające na manipulację obiektami DataFrame

Wykorzystajmy następujące funkcje:
- `unionAll()`: łączy ze sobą dwa obiekty DataFrame
- `orderBy()`: wykonaje sortowanie kolumn obiektu DataFrame
- `select()`: wybiera, które kolumny DataFrame mają pozostać do dalszych manipulacji nimi
- `drop()`: wybiera jedną kolumnę DataFrame do usunięcia
- `filter()`: zachowuje wiersze DataFrame pasujące do podanego warunku

In [27]:
# Pobierz ramkę DataFrame i dodaj ją do samej siebie
nice_df \
 .unionAll(nice_df) \
 .show() # zwróć uwagę na zapis - użycie nawiasów

# Dodaj to do siebie dwukrotnie
(nice_df
 .unionAll(nice_df)
 .unionAll(nice_df)
 .show())

# Kolumny zostały zamienione kolejnością
# dlatego jeśli schamt łączonych DF nie zgadza się nastąpi wymuszenie domyślnego typu
(nice_df
 .select(['numbers', 'letters'])
 .unionAll(nice_df)
 .show())

(nice_df
 .select(['numbers', 'letters'])
 .unionAll(nice_df)
 .printSchema())

+-------+-------+
|letters|numbers|
+-------+-------+
|      a|      1|
|      b|      2|
|      c|      3|
|      a|      1|
|      b|      2|
|      c|      3|
+-------+-------+

+-------+-------+
|letters|numbers|
+-------+-------+
|      a|      1|
|      b|      2|
|      c|      3|
|      a|      1|
|      b|      2|
|      c|      3|
|      a|      1|
|      b|      2|
|      c|      3|
+-------+-------+

+-------+-------+
|numbers|letters|
+-------+-------+
|      1|      a|
|      2|      b|
|      3|      c|
|      a|      1|
|      b|      2|
|      c|      3|
+-------+-------+

root
 |-- numbers: string (nullable = true)
 |-- letters: string (nullable = true)



In [28]:
# Sortowanie DataFrame względem kolumny `numbers`
(nice_df
 .unionAll(nice_df)
 .unionAll(nice_df)
 .orderBy('numbers')
 .show())

# Posortuj tę samą kolumnę w odwrotnej kolejności
(nice_df
 .unionAll(nice_df)
 .unionAll(nice_df)
 .orderBy('numbers',
          ascending = False)
 .show())

+-------+-------+
|letters|numbers|
+-------+-------+
|      a|      1|
|      a|      1|
|      a|      1|
|      b|      2|
|      b|      2|
|      b|      2|
|      c|      3|
|      c|      3|
|      c|      3|
+-------+-------+

+-------+-------+
|letters|numbers|
+-------+-------+
|      c|      3|
|      c|      3|
|      c|      3|
|      b|      2|
|      b|      2|
|      b|      2|
|      a|      1|
|      a|      1|
|      a|      1|
+-------+-------+



In [29]:
# `select ()` i `drop ()` obie pobierają jako argumenty listę nazw kolumn
# a same funkcje robią dokładnie to, czego można się po ich nazwach spodziewać ;)

# Wybierz tylko pierwszą kolumnę DF
(nice_df
 .select('letters')
 .show())

# Zmień kolejność kolumn w DF za pomocą `select()`
(nice_df
 .select(['numbers', 'letters'])
 .show())

# Usuń drugą kolumnę DF
(nice_df
 .drop('letters')
 .show())

+-------+
|letters|
+-------+
|      a|
|      b|
|      c|
+-------+

+-------+-------+
|numbers|letters|
+-------+-------+
|      1|      a|
|      2|      b|
|      3|      c|
+-------+-------+

+-------+
|numbers|
+-------+
|      1|
|      2|
|      3|
+-------+



In [30]:
# Funkcja `filter()` wykonuje filtrowanie wierszy DF

# Oto kilka filtrów numerycznych z operatorami porównania
# (>, <, >=, <=, ==, != )

# Filtruj wiersze, dla których wartości kolumny "numbers" jest większa od 1
(nice_df
 .filter(nice_df.numbers > 1)
 .show())

# Wykonaj dwie operacje filtrowania
(nice_df
 .filter(nice_df.numbers > 1)
 .filter(nice_df.numbers < 3)
 .show())

# Nie tylko liczby! Użyj `filter ()` + `isin ()` 
# połączonych razem w celu filtrowania kolumn zawiarających wartości typu string
(nice_df
 .filter(nice_df.letters
         .isin(['a', 'b']))
 .show())

+-------+-------+
|letters|numbers|
+-------+-------+
|      b|      2|
|      c|      3|
+-------+-------+

+-------+-------+
|letters|numbers|
+-------+-------+
|      b|      2|
+-------+-------+

+-------+-------+
|letters|numbers|
+-------+-------+
|      a|      1|
|      b|      2|
+-------+-------+



### Funkcja 'groupBy' i agragatory

Funkcja `groupBy ()` grupuje elementy DataFrame za pomocą określonych kolumn, a następnie można uruchamiać różnego rodzaju agregacje. Dostępne funkcje agregujące:

- `count()`: zlicza liczbę rekordów dla każdej grupy
- `sum()`: oblicza sumę dla każdej kolumny liczbowej dla każdej grupy
- `min()`: wyznacza minimalną wartość dla każdej kolumny numerycznej dla każdej z grup
- `max()`: wyznacza maksymalną wartość dla każdej kolumny numerycznej dla każdej z grup
- `avg()` lub `mean()`: oblicza średnie wartości dla każdej kolumny numerycznej dla każdej grupy
- `pivot()`: obraca kolumnę bieżącej DataFrame i wykonuje określoną agregację

Zanim przejdziemy do agregacji, wczytajmy **CSV** z bardziej interesującymi danymi i stworzymy nowy obiekt DataFrame.

Zbiór danych, który zostanie załadowany zawiera dane dotyczące lotów z lotnisk w Nowym Jorku (`JFK`,` LGA`, `EWR`) w 2013 r. Ma 336 776 wierszy i 16 kolumn.

In [0]:
# Utwórz obiekt schematu ...
nycflights_schema = StructType([
  StructField('year', IntegerType(), True),
  StructField('month', IntegerType(), True),
  StructField('day', IntegerType(), True),
  StructField('dep_time', StringType(), True),
  StructField('dep_delay', IntegerType(), True),
  StructField('arr_time', StringType(), True),
  StructField('arr_delay', IntegerType(), True),
  StructField('carrier', StringType(), True),
  StructField('tailnum', StringType(), True),
  StructField('flight', StringType(), True),  
  StructField('origin', StringType(), True),
  StructField('dest', StringType(), True),
  StructField('air_time', IntegerType(), True),
  StructField('distance', IntegerType(), True),
  StructField('hour', IntegerType(), True),
  StructField('minute', IntegerType(), True)
  ])

# ... a następnie odczytaj plik CSV korzystając ze zdefiniowanego schematu
nycflights = (sqlContext
              .read
              .format('com.databricks.spark.csv')
              .schema(nycflights_schema)
              .options(header = True)
              .load('/content/colab-examples/spark/data/nycflights13.csv'))

In [34]:
# Zapoznaj się ze schematem importowanego zbioru danych
nycflights.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)



In [36]:
# W przypadku notebooków Databricks Cloud dostępna jest funkcja `display ()` pozwalająca "ładnie"
# wyświetlić objekty typu DataFrame

# display(nycflights)

DataFrame[year: int, month: int, day: int, dep_time: string, dep_delay: int, arr_time: string, arr_delay: int, carrier: string, tailnum: string, flight: string, origin: string, dest: string, air_time: int, distance: int, hour: int, minute: int]

In [37]:
# W naszym przypadku skorzystamy z funkcji `show()` i wyświetlimy pierwszych 5 rekordów
nycflights.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2013|    1|  1|     517|        2|     830|       11|     UA| N14228|  1545|   EWR| IAH|     227|    1400|   5|    17|
|2013|    1|  1|     533|        4|     850|       20|     UA| N24211|  1714|   LGA| IAH|     227|    1416|   5|    33|
|2013|    1|  1|     542|        2|     923|       33|     AA| N619AA|  1141|   JFK| MIA|     160|    1089|   5|    42|
|2013|    1|  1|     544|       -1|    1004|      -18|     B6| N804JB|   725|   JFK| BQN|     183|    1576|   5|    44|
|2013|    1|  1|     554|       -6|     812|      -25|     DL| N668DN|   461|   LGA| ATL|     116|     762|   5|    54|
+----+-----+---+--------+---------+-----

In [38]:
# Pogrupujmy i agregujmy dane

#`groupBy ()` grupuje jedną lub więcej kolumn DF 
# i przygotowuje je dalszych operacji za pomocą funkcji agregujących
(nycflights
 .groupby('month') # tworzy zgrupowane dane
 .count() # tworzy nową kolumnę zawierającą wynik działania funkcji `count`
 .show())

# Użyj funkcji `agg()`, aby wykonać wiele agregacji

(nycflights
 .groupby('month')
 .agg({'dep_delay': 'avg', 'arr_delay': 'avg'}) # zauważ jakie są nazwy nowych kolumn
 .show())

# Zastrzeżenie: nie można wykonać wielu agregacji 
# w tej samej kolumnie (wykonywana jest tylko ostatnia)

(nycflights
 .groupby('month')
 .agg({'dep_delay': 'min', 'dep_delay': 'max'})
 .show())

+-----+-----+
|month|count|
+-----+-----+
|   12|28135|
|    1|27004|
|    6|28243|
|    3|28834|
|    5|28796|
|    9|27574|
|    4|28330|
|    8|29327|
|    7|29425|
|   10|28889|
|   11|27268|
|    2|24951|
+-----+-----+

+-----+--------------------+------------------+
|month|      avg(arr_delay)|    avg(dep_delay)|
+-----+--------------------+------------------+
|   12|  14.870355292376018|16.576687569162672|
|    1|   6.129971967573301|10.036665030396858|
|    6|  16.481329639889196|20.846331791143424|
|    3|   5.807576517812343|13.227076109105209|
|    5|   3.521508816837315|12.986859348988771|
|    9|  -4.018363569048501|6.7224762185679525|
|    4|  11.176062980699463|13.938037741305763|
|    8|   6.040652385589095|12.611039839117922|
|    7|  16.711306683631992|21.727786554326837|
|   10|-0.16706268781885528| 6.243988413080655|
|   11|  0.4613473731044455|  5.43536156833734|
|    2|   5.613019355385202|10.816842549598986|
+-----+--------------------+------------------+

+-----

In [39]:
# Użyj `groupBy()` z kilkoma kolumnami, a następnie agreguj wartości
(nycflights
  .groupby(['month', 'origin', 'dest']) # pogrupuj według tych unikalnych kombinacji
  .count()                              # a następnie wykonaj Wykonaj agregację "count" na grupach
  .orderBy(['month', 'count'],
           ascending = [1, 0]) # uporządkuj według kolumny `month` rosnąco, a kolumnę `count` malejąco
  .show()
) 

+-----+------+----+-----+
|month|origin|dest|count|
+-----+------+----+-----+
|    1|   JFK| LAX|  937|
|    1|   LGA| ATL|  878|
|    1|   JFK| SFO|  671|
|    1|   LGA| ORD|  583|
|    1|   EWR| ORD|  502|
|    1|   JFK| BOS|  486|
|    1|   JFK| MCO|  456|
|    1|   LGA| MIA|  451|
|    1|   JFK| FLL|  439|
|    1|   LGA| DFW|  437|
|    1|   LGA| CLT|  437|
|    1|   EWR| BOS|  430|
|    1|   LGA| DTW|  429|
|    1|   EWR| MCO|  422|
|    1|   JFK| SJU|  411|
|    1|   EWR| CLT|  381|
|    1|   EWR| FLL|  370|
|    1|   EWR| ATL|  362|
|    1|   LGA| DCA|  361|
|    1|   LGA| FLL|  352|
+-----+------+----+-----+
only showing top 20 rows



In [40]:
# Użyj `groupBy ()` + `pivot ()` + funkcji agregacji, 
# aby utworzyć tabelę przestawną!

# Utwórz tabelę lotów według miesiąca dla każdego przewoźnika
(
  nycflights
  .groupBy('month') # pogrupuj dane względem kolumny 'month'
  .pivot('carrier') # ustaw kolumny danych korzystając za skrótów przewoźników z kolumny `carrier`
  .count()          # twórz agregacje jako licznik wierszy
  .orderBy(['month'],
           ascending = [1])
  .show()
)

+-----+----+----+---+----+----+----+---+---+---+----+----+----+----+---+----+---+
|month|  9E|  AA| AS|  B6|  DL|  EV| F9| FL| HA|  MQ|  OO|  UA|  US| VX|  WN| YV|
+-----+----+----+---+----+----+----+---+---+---+----+----+----+----+---+----+---+
|    1|1573|2794| 62|4427|3690|4171| 59|328| 31|2271|   1|4637|1602|316| 996| 46|
|    2|1459|2517| 56|4103|3444|3827| 49|296| 28|2044|null|4346|1552|271| 911| 48|
|    3|1627|2787| 62|4772|4189|4726| 57|316| 31|2256|null|4971|1721|303| 998| 18|
|    4|1511|2722| 60|4517|4092|4561| 57|311| 30|2211|null|5047|1727|466| 980| 38|
|    5|1462|2803| 62|4576|4082|4817| 58|325| 31|2284|null|4960|1785|496|1006| 49|
|    6|1437|2757| 60|4622|4126|4456| 55|252| 30|2178|   2|4975|1736|480|1028| 49|
|    7|1494|2882| 62|4984|4251|4641| 58|263| 31|2261|null|5066|1786|489|1076| 81|
|    8|1456|2856| 62|4952|4318|4563| 55|263| 31|2263|   4|5124|1779|489|1047| 65|
|    9|1540|2614| 60|4291|3883|4725| 58|255| 25|2206|  20|4694|1698|453|1010| 42|
|   10|1673|2715

In [41]:
# Kolejny pomysł tabeli przestawnej: uzyskaj średnie opóźnienie 
# odlotu dla każdego przewoźnika na różnych lotniskach NYC
(
  nycflights
  .groupBy('carrier')
  .pivot('origin')
  .avg('dep_delay')
  .show()
)

+-------+------------------+------------------+------------------+
|carrier|               EWR|               JFK|               LGA|
+-------+------------------+------------------+------------------+
|     UA| 12.52286865854727|               7.9|12.087916294500447|
|     AA|10.035419126328216|10.302155109221522| 6.705769103100312|
|     EV| 20.16493117893477|18.520361990950228| 19.12549969715324|
|     B6|13.100262224278882|12.757453126122458|14.805738396624472|
|     DL|12.084592145015106| 8.333187709334497|  9.57299733123332|
|     OO|20.833333333333332|              null|10.434782608695652|
|     F9|              null|              null|20.215542521994134|
|     YV|              null|              null|18.996330275229358|
|     US| 3.735103926096998| 5.866958571909734|3.3065054875139177|
|     MQ|17.467267552182165|13.199970870958346| 8.528568781271234|
|     HA|              null| 4.900584795321637|              null|
|     AS| 5.804775280898877|              null|              n

### Operacje na kolumnach

Instancje obiektu `Column` mogą być tworzone przez:

(1) Wybór kolumny z DataFrame
- `df.colName`
- `df["colName"]`
- `df.select(df.colName)`
- `df.withColumn(df.colName)`

(2) Korzystając z wyrażenia
- `df.colName + 1`
- `1 / df.colName`

Po utworzeniu instancji `Column` można zastosować szeroką gamę funkcji. Niektóre z omawianych dalej funkcji to:
- `format_number()`: zastosuj formatowanie liczby zaokrąglonej do "d" miejsc dziesiętnych i zwróć wynik jako ciąg znaków
- `when()` & `otherwise()`: `when()` przetwórz listę warunków i zwróć jedno z wielu możliwych wyrażeń wynikowych; jeśli `otherwise()` nie zostanie użyte, zwracane jest `None` dla nieokreślonych warunków 
- `concat_ws()`: łączy wiele kolumn wejściowych ciągów znaków w jedną kolumnę ciągów, używając danego separatora
- `to_utc_timestamp()`: zakłada, że dany znacznik czasu jest w zadanej strefie czasowej i konwertuje na UTC
- `year()`: wyodrębnia rok z określonej daty jako liczbę całkowitą
- `month()`: wyodrębnia miesiąc z określonej daty jako liczbę całkowitą
- `dayofmonth()`: wyodrębnia dzień miesiąca danej daty jako liczbę całkowitą
- `hour()`: wyodrębnia godzinę z określonej daty jako liczbę całkowitą
- `minute()`: wyodrębnia minutę podanej daty jako liczbę całkowitą

In [42]:
# Wykonaj 2 różne agregacje, zmień nazwy tych nowych kolumn, 
# a następnie wykonaj zaokrąglenie wartości zagregowanych
(
  nycflights
  .groupby('month')
  .agg({'dep_delay': 'avg', 'arr_delay': 'avg'})
  .withColumnRenamed('avg(arr_delay)', 'mean_arr_delay')
  .withColumnRenamed('avg(dep_delay)', 'mean_dep_delay')
  .withColumn('mean_arr_delay', format_number('mean_arr_delay', 1)) #do 1 miejsca po przecinku
  .withColumn('mean_dep_delay', format_number('mean_dep_delay', 1))
  .orderBy(['month'],
           ascending = [1])
  .show()
)

+-----+--------------+--------------+
|month|mean_arr_delay|mean_dep_delay|
+-----+--------------+--------------+
|    1|           6.1|          10.0|
|    2|           5.6|          10.8|
|    3|           5.8|          13.2|
|    4|          11.2|          13.9|
|    5|           3.5|          13.0|
|    6|          16.5|          20.8|
|    7|          16.7|          21.7|
|    8|           6.0|          12.6|
|    9|          -4.0|           6.7|
|   10|          -0.2|           6.2|
|   11|           0.5|           5.4|
|   12|          14.9|          16.6|
+-----+--------------+--------------+



In [43]:
# Dodaj nową kolumnę (`far_or_near`) zawierającej typ string zbudowanym na porównaniu kolumny 
# numerycznej 'distance'; użyj: `withColumn()`, `when()` oraz `otherwise()`
(
  nycflights
  .withColumn('far_or_near',
              when(nycflights.distance > 1000, 'far') # składnia `if-then`
              .otherwise('near'))                     # składnia `else`
  .show()
)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|far_or_near|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------+
|2013|    1|  1|     517|        2|     830|       11|     UA| N14228|  1545|   EWR| IAH|     227|    1400|   5|    17|        far|
|2013|    1|  1|     533|        4|     850|       20|     UA| N24211|  1714|   LGA| IAH|     227|    1416|   5|    33|        far|
|2013|    1|  1|     542|        2|     923|       33|     AA| N619AA|  1141|   JFK| MIA|     160|    1089|   5|    42|        far|
|2013|    1|  1|     544|       -1|    1004|      -18|     B6| N804JB|   725|   JFK| BQN|     183|    1576|   5|    44|        far|
|2013|    1|  1|     554|       -6|     812|      -25|     DL| N668DN|   461

In [44]:
# Wykonaj kilka obliczeń numerycznych na kolumnach
(
  nycflights
  .withColumn('dist_per_minute',
              nycflights.distance / nycflights.air_time) # utwórz nową kolumnę zawierającą dzilenie wartości
  .withColumn('dist_per_minute',
              format_number('dist_per_minute', 2))       # zaokrąglij wartośći z nowej kolumny do dwóch miejsc po przecinku
  .drop('distance') # usuń kolumnę `distance`
  .drop('air_time') # usuń kolumnę `air_time`
  .show()
)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+----+------+---------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|hour|minute|dist_per_minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+----+------+---------------+
|2013|    1|  1|     517|        2|     830|       11|     UA| N14228|  1545|   EWR| IAH|   5|    17|           6.17|
|2013|    1|  1|     533|        4|     850|       20|     UA| N24211|  1714|   LGA| IAH|   5|    33|           6.24|
|2013|    1|  1|     542|        2|     923|       33|     AA| N619AA|  1141|   JFK| MIA|   5|    42|           6.81|
|2013|    1|  1|     544|       -1|    1004|      -18|     B6| N804JB|   725|   JFK| BQN|   5|    44|           8.61|
|2013|    1|  1|     554|       -6|     812|      -25|     DL| N668DN|   461|   LGA| ATL|   5|    54|           6.57|
|2013|    1|  1|     554|       -4|     740|       12|  

In [47]:
# Stwórz sygnaturę czasową
# Mamy wszystkie komponenty: `year`,` month`, `day`,` hour` i `minute`

# Użyj `concat_ws ()` (połącz z separatorem), aby połączyć dane kolumnowe 
# w kolumny typu StringType, tak, że tworzone są daty (separator `-`, YYYY-MM-DD) 
# i czasy (separator` `:`, czas 24-godzinny)

nycflights = \
(nycflights
 .withColumn('date',
             concat_ws('-',
                       nycflights.year,
                       nycflights.month,
                       nycflights.day))
 .withColumn('time',
             concat_ws(':',
                       nycflights.hour,
                       nycflights.minute)))

# W drugim kroku należy połączyć za pomocą `concat_ws()` łańcuchy znaków 
# z kolumn `date` i` time` (separatorem jest spacją); następnie usunąć niepotrzebne kolumny

cols_drop = ['year', 'month', 'day', 'hour', 'minute', 'time', 'date']
nycflights = \
(nycflights
 .withColumn('timestamp',
             concat_ws(' ',
                       nycflights.date,
                       nycflights.time))
 .drop(*cols_drop)     # `drop()` akceptuje listę kolumn, ale musimy ją rozpakować
)  


#nycflights = \
#(nycflights
# .withColumn('timestamp',
#             concat_ws(' ',
#                       nycflights.date,
#                       nycflights.time))
# .drop('year')     # tak było: `drop()` nie akceptuje jako argumentu
# .drop('month')    # listy kolumn do usunięcia
# .drop('day')      # dlatego, dla każdej kolumny,
# .drop('hour')     # którą chcemy usunąć
# .drop('minute')   # z obiektu DataFrame,  
# .drop('date')     # musimy stworzyć nową 
# .drop('time'))    # komendę `drop()`

# W ostatnim kroku skonwertuj typ  kolumny `timestamp` z StringType 
# na TimestampType
nycflights = \
(nycflights
  .withColumn('timestamp',
               to_utc_timestamp(nycflights.timestamp, 'GMT'))
)

AttributeError: ignored

In [48]:
# Wyświetl DataFrame `nycflights` i jego kolumnę `timestamp`
nycflights.show()

+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|          timestamp|
+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|     517|        2|     830|       11|     UA| N14228|  1545|   EWR| IAH|     227|    1400|2013-01-01 05:17:00|
|     533|        4|     850|       20|     UA| N24211|  1714|   LGA| IAH|     227|    1416|2013-01-01 05:33:00|
|     542|        2|     923|       33|     AA| N619AA|  1141|   JFK| MIA|     160|    1089|2013-01-01 05:42:00|
|     544|       -1|    1004|      -18|     B6| N804JB|   725|   JFK| BQN|     183|    1576|2013-01-01 05:44:00|
|     554|       -6|     812|      -25|     DL| N668DN|   461|   LGA| ATL|     116|     762|2013-01-01 05:54:00|
|     554|       -4|     740|       12|     UA| N39463|  1696|   EWR| ORD|     150|     719|2013

In [0]:
# Tak naprawdę nie ma to znaczenia, ale dla porządku, wskazane jest aby  kolumna `timestamp` 
# była pierwszą kolumną; zastosujmy metodę `columns` i plastrowanie (ang. slicing) :)

nycflights = \
 (nycflights
  .select(nycflights.columns[-1:] + nycflights.columns[0:-1])) # funkcja `columns` zwraca listę nazw kolumn, więc -1:

In [51]:
# Teraz kolumna `timestamp` jest pierwszą kolumną:
nycflights.show()

+--------+-------------------+--------+---------+--------+---------+-------+-------+------+------+----+--------+
|distance|          timestamp|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|
+--------+-------------------+--------+---------+--------+---------+-------+-------+------+------+----+--------+
|    1400|2013-01-01 05:17:00|     517|        2|     830|       11|     UA| N14228|  1545|   EWR| IAH|     227|
|    1416|2013-01-01 05:33:00|     533|        4|     850|       20|     UA| N24211|  1714|   LGA| IAH|     227|
|    1089|2013-01-01 05:42:00|     542|        2|     923|       33|     AA| N619AA|  1141|   JFK| MIA|     160|
|    1576|2013-01-01 05:44:00|     544|       -1|    1004|      -18|     B6| N804JB|   725|   JFK| BQN|     183|
|     762|2013-01-01 05:54:00|     554|       -6|     812|      -25|     DL| N668DN|   461|   LGA| ATL|     116|
|     719|2013-01-01 05:54:00|     554|       -4|     740|       12|     UA| N39463|  1696|   EW

In [52]:
# Sprawdź schemat DataFrame, zauważ, że kolumna `timestamp` 
# jest rzeczywiście klasyfikowana jako typ znacznika czasu

nycflights.printSchema()

root
 |-- distance: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)



In [53]:
# Jeśli brakuje nam kolumn ze składowymi czasu, możemy je odzyskać!
# Używaj funkcji `year ()`, `month ()`, `dayofmonth ()`, `hour ()` 
# oraz `minute ()` z funkcją `withColumn ()`

(
  nycflights
  .withColumn('year', year(nycflights.timestamp))
  .withColumn('month', month(nycflights.timestamp))
  .withColumn('day', dayofmonth(nycflights.timestamp))
  .withColumn('hour', hour(nycflights.timestamp))
  .withColumn('minute', minute(nycflights.timestamp))
  .show()
)

+--------+-------------------+--------+---------+--------+---------+-------+-------+------+------+----+--------+----+-----+---+----+------+
|distance|          timestamp|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|year|month|day|hour|minute|
+--------+-------------------+--------+---------+--------+---------+-------+-------+------+------+----+--------+----+-----+---+----+------+
|    1400|2013-01-01 05:17:00|     517|        2|     830|       11|     UA| N14228|  1545|   EWR| IAH|     227|2013|    1|  1|   5|    17|
|    1416|2013-01-01 05:33:00|     533|        4|     850|       20|     UA| N24211|  1714|   LGA| IAH|     227|2013|    1|  1|   5|    33|
|    1089|2013-01-01 05:42:00|     542|        2|     923|       33|     AA| N619AA|  1141|   JFK| MIA|     160|2013|    1|  1|   5|    42|
|    1576|2013-01-01 05:44:00|     544|       -1|    1004|      -18|     B6| N804JB|   725|   JFK| BQN|     183|2013|    1|  1|   5|    44|
|     762|2013-01-01

Jest więcej funkcji operujących na danych czasowych:
- `date_sub()`: odejmij liczbę całkowitą dni od *Date* lub *Timestamp*
- `date_add()`: dodaj liczbę całkowitą dni do *Date* lub *Timestamp*
- `datediff()`: określ różnicę między dwiema datami
- `add_months()`: dodaj liczbę całkowitą miesięcy
- `months_between()`: określ liczbę miesięcy między dwiema datami
- `next_day()`: zwraca pierwszą datę, która jest późniejsza niż wartość kolumny daty
- `last_day()`: zwraca ostatni dzień miesiąca, do którego należy dana data
- `dayofmonth()`: wyodrębnij dzień miesiąca danej daty jako liczbę całkowitą
- `dayofyear()`: wyodrębnij dzień roku danej daty jako liczbę całkowitą
- `weekofyear()`: wyodrębnij numer tygodnia danej daty jako liczbę całkowitą
- `quarter()`: wyodrębnij kwartał danej daty

In [54]:
# Przetwórzmy znacznik czasu z pierwszego rekordu `nycflights` 
# z każdą z wyżej wymienionych funkcji
(
  nycflights
   .limit(1) # stąd pierwszy i jedyny
   .select('timestamp')
   .withColumn('date_sub', date_sub(nycflights.timestamp, 7)) #odejmij tydzień
   .withColumn('date_add', date_add(nycflights.timestamp, 7)) # dodaj 7 dni
   .withColumn('datediff', datediff(nycflights.timestamp, nycflights.timestamp)) #odejmij od daty tę samą datę
   .withColumn('add_months', add_months(nycflights.timestamp, 1)) # dodaj 1 miesiąc
   .withColumn('months_between', months_between(nycflights.timestamp, nycflights.timestamp)) # jaka jest róznica mięsięcy? 
   .withColumn('next_day', next_day(nycflights.timestamp, 'Mon')) #podaj datę następnego poniedziałka
   .withColumn('last_day', last_day(nycflights.timestamp)) # ostatni dzień miesiąca dla daty
   .withColumn('dayofmonth', dayofmonth(nycflights.timestamp)) #który to dzień miesiąca
   .withColumn('dayofyear', dayofyear(nycflights.timestamp)) # który dzień roku
   .withColumn('weekofyear', weekofyear(nycflights.timestamp)) # który tydzień w roku
   .withColumn('quarter', quarter(nycflights.timestamp)) #który kwartał
   .toPandas() #ładniej wygląda ;)
   )

,timestamp,date_sub,date_add,datediff,add_months,months_between,next_day,last_day,dayofmonth,dayofyear,weekofyear,quarter
0,2013-01-01 05:17:00,2012-12-25,2013-01-08,0,2013-02-01,0.0,2013-01-07,2013-01-31,1,1,1,1


### Złączenia

Złączenia można łatwo wykonać za pomocą Spark DataFrames. Jest to wyrażenie postaci:

`join(other, on = None, how = None)`

gdzie:
- other: DataFrame, który służy jako prawa stronie złączenia
- on: zazwyczaj wyrażenie złączenia
- how: domyślnie jest to `inner`, ale oprócz `inner`są również dostępne opcje `outer`, `left_outer`, `right_outer` oraz `leftsemi`

Załadujmy więcej danych, abyśmy mogli złączyć do siebie dwa obiekty DataFrame. Plik **CSV** `weather.csv` zawiera godzinowe dane meteorologiczne z portu lotniczego `Newark Liberty International Airport` (już znany nam EWR), zarejtrowane w 2013 roku.

In [0]:
# Stwórz schemat danych...
weather_schema = StructType([  
  StructField('year', IntegerType(), True),
  StructField('month', IntegerType(), True),
  StructField('day', IntegerType(), True),
  StructField('hour', IntegerType(), True),
  StructField('temp', FloatType(), True),
  StructField('dewp', FloatType(), True),
  StructField('humid', FloatType(), True),
  StructField('wind_dir', IntegerType(), True),
  StructField('wind_speed', FloatType(), True),
  StructField('wind_gust', FloatType(), True),
  StructField('precip', FloatType(), True),
  StructField('pressure', FloatType(), True),
  StructField('visib', FloatType(), True)
  ])

#...i wczytaj dane z uwzględnieniem ich schematu
weather = \
(sqlContext
 .read
 .format('com.databricks.spark.csv')
 .schema(weather_schema)
 .options(header = True)
 .load('/content/colab-examples/spark/data/weather.csv'))

In [57]:
# Spójrz na zaimportowany zestaw danych
weather.limit(5).toPandas()

,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib
0,2013,1,1,0,37.040001,21.92,53.970001,230,10.35702,11.918652,0.0,1013.900024,10.0
1,2013,1,1,1,37.040001,21.92,53.970001,230,13.80936,15.891536,0.0,1013.000000,10.0
2,2013,1,1,2,37.939999,21.92,52.090000,230,12.65858,14.567241,0.0,1012.599976,10.0
3,2013,1,1,3,37.939999,23.00,54.509998,230,13.80936,15.891536,0.0,1012.700012,10.0
4,2013,1,1,4,37.939999,24.08,57.040001,240,14.96014,17.215830,0.0,1012.799988,10.0


In [0]:
# Potrzebujemy ponownie wartości `month`,` day` i `hour`
nycflights = \
(nycflights
 .withColumn('month', month(nycflights.timestamp))
 .withColumn('day', dayofmonth(nycflights.timestamp))
 .withColumn('hour', hour(nycflights.timestamp)))

# Dołącz do obiektu `nycflights` nowowczytany obiekt `weather`
nycflights_all_columns = \
(nycflights
 .join(weather,
       [nycflights.month == weather.month, # trzy warunki złączenia: month,
        nycflights.day == weather.day,     #                         day,
        nycflights.hour == weather.hour],  #                         hour
       'left_outer')) # left outer join: zachowaj wszystkie wiersze z lewego DF (flights), 
                      # z pasującymi wierszami z prawej DF (weather), 
                      # wartości NULL będą utworzone, jeśli nie ma pasująchy do prawego DF  

In [61]:
# Zauważ, że wiele kolumn zostało utworzonych, a także zduplikowane nazwy kolumn (nie błąd! to jest funkcjonalność)
nycflights_all_columns.show()

+--------+-------------------+--------+---------+--------+---------+-------+-------+------+------+----+--------+-----+---+----+----+-----+----+----+-----+-----+-----+--------+----------+---------+------+--------+-----+
|distance|          timestamp|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|month|day|hour|year|month| day|hour| temp| dewp|humid|wind_dir|wind_speed|wind_gust|precip|pressure|visib|
+--------+-------------------+--------+---------+--------+---------+-------+-------+------+------+----+--------+-----+---+----+----+-----+----+----+-----+-----+-----+--------+----------+---------+------+--------+-----+
|    1400|2013-01-01 05:17:00|     517|        2|     830|       11|     UA| N14228|  1545|   EWR| IAH|     227|    1|  1|   5|null| null|null|null| null| null| null|    null|      null|     null|  null|    null| null|
|    1416|2013-01-01 05:33:00|     533|        4|     850|       20|     UA| N24211|  1714|   LGA| IAH|     227|    1|  1|  

In [0]:
# Jednym ze sposobów zmniejszenia liczby 
# nieistotnych kolumn jest użycie instrukcji `select()`
nycflights_wind_visib = \
(nycflights_all_columns
 .select(['timestamp', 'carrier', 'flight',
          'origin', 'dest', 'wind_dir',
          'wind_speed', 'wind_gust', 'visib']))

In [63]:
# Przejrzyj DataFrame, teraz z mniejszą ilością kolumn
nycflights_wind_visib.limit(5).toPandas()

,timestamp,carrier,flight,origin,dest,wind_dir,wind_speed,wind_gust,visib
0,2013-01-01 05:17:00,UA,1545,EWR,IAH,None,NaN,NaN,NaN
1,2013-01-01 05:33:00,UA,1714,LGA,IAH,None,NaN,NaN,NaN
2,2013-01-01 05:42:00,AA,1141,JFK,MIA,None,NaN,NaN,NaN
3,2013-01-01 05:44:00,B6,725,JFK,BQN,None,NaN,NaN,NaN
4,2013-01-01 05:54:00,DL,461,LGA,ATL,None,NaN,NaN,NaN


Załadujmy jeszcze więcej danych, abyśmy mogli ustalić, czy w bardzo wietrzną pogodę wystąpiły jakieś starty.

**CSV** `beaufort_land.csv` zawiera wartości skali Beauforta (kolumna` force`), zakresy prędkości wiatru są okraślane w *mph* i nazwą dla każdej siły wiatru.

In [0]:
# Stwórz schemat danych... 
beaufort_land_schema = StructType([  
  StructField('force', IntegerType(), True),
  StructField('speed_mi_h_lb', IntegerType(), True),
  StructField('speed_mi_h_ub', IntegerType(), True),
  StructField('name', StringType(), True)
  ])

# ...itd...
beaufort_land = \
(sqlContext
 .read
 .format('com.databricks.spark.csv')
 .schema(beaufort_land_schema)
 .options(header = True)
 .load('/content/colab-examples/spark/data/beaufort_land.csv'))

In [66]:
# Rzuć okiem na dane
beaufort_land.limit(5).toPandas()

,force,speed_mi_h_lb,speed_mi_h_ub,name
0,0,0,1,calm
1,1,1,4,light air
2,2,5,7,light breeze
3,3,8,11,gentle breeze
4,4,12,18,moderate breeze


In [0]:
# Złacz obecny DF i obiekt `beaufort_land` jako warunek złączenia 
# użyj wyrażenia zawierającego przedziały siły wiatru
nycflights_wind_visib_beaufort = \
(nycflights_wind_visib
 .join(beaufort_land,
      [nycflights_wind_visib.wind_speed >= beaufort_land.speed_mi_h_lb,
       nycflights_wind_visib.wind_speed < beaufort_land.speed_mi_h_ub],
       'left_outer')
 .withColumn('month', month(nycflights_wind_visib.timestamp)) # Utwórz kolumnę o nazwie `month` z wartości kolumny `timestamp`
 .drop('speed_mi_h_lb')
 .drop('speed_mi_h_ub')
)

In [68]:
# View the joined DF; now we have extra data on wind speed!
nycflights_wind_visib_beaufort.show()

+-------------------+-------+------+------+----+--------+----------+---------+-----+-----+-------------+-----+
|          timestamp|carrier|flight|origin|dest|wind_dir|wind_speed|wind_gust|visib|force|         name|month|
+-------------------+-------+------+------+----+--------+----------+---------+-----+-----+-------------+-----+
|2013-01-01 05:17:00|     UA|  1545|   EWR| IAH|    null|      null|     null| null| null|         null|    1|
|2013-01-01 05:33:00|     UA|  1714|   LGA| IAH|    null|      null|     null| null| null|         null|    1|
|2013-01-01 05:42:00|     AA|  1141|   JFK| MIA|    null|      null|     null| null| null|         null|    1|
|2013-01-01 05:44:00|     B6|   725|   JFK| BQN|    null|      null|     null| null| null|         null|    1|
|2013-01-01 05:54:00|     DL|   461|   LGA| ATL|    null|      null|     null| null| null|         null|    1|
|2013-01-01 05:54:00|     UA|  1696|   EWR| ORD|    null|      null|     null| null| null|         null|    1|
|

In [0]:
# Możemy sprawdzić liczbę potencjalnie niebezpiecznych startów (tj. gdy siła Beauforta jest wysoka)
# w każdym miesiącu poprzez użycie funkcji `crosstab()`
crosstab_month_force = \
  (nycflights_wind_visib_beaufort
      .crosstab('month', 'force'))

# Po utworzeniu tabeli crosstab_month_force, 
# użyj kilku funkcji do jej oczyszczenia
crosstab_month_force = \
(crosstab_month_force
 .withColumn('month_force',
             crosstab_month_force.month_force.cast('int')) # kolumna jest początkowo typu string, 
                                                           # ale przekształcenie na "int" pomoże uporządkować jej kolejność 
 .orderBy('month_force')                                   # w następnym wyrażeniu
 .drop('null'))

crosstab_month_force = \
(crosstab_month_force
 .filter(crosstab_month_force.
         month_force.
         isNotNull()))

In [71]:
# Otrzymaliśmy tabelę krzyżową; Okazuje się, że styczeń był nieco bardziej ryzykowny dla startów z powodu silnego wiatru
crosstab_month_force.show()

+-----------+----+----+----+----+----+----+---+---+---+
|month_force|   0|   1|   2|   3|   4|   5|  6|  7|  8|
+-----------+----+----+----+----+----+----+---+---+---+
|          1|2101|1261|5100|6442|5233|1161|521|164| 92|
|          2|1852|1512|4525|5254|5490|2190|664|  0|  0|
|          3|1126|1228|3874|6774|8232|2180|543|  0|  0|
|          4|2050|1891|4921|6653|6575|1252| 65|  0|  0|
|          5|2758|1736|6740|7408|3709| 736|217| 38|  0|
|          6|1393|1911|5259|7710|5825| 704| 18| 39|  0|
|          7|1530|1742|4908|9076|6415| 343|  0|  0|  0|
|          8|2999|3305|6243|7147|3949| 203|  0|  0|  0|
|          9|1729|2142|6050|8885|3663| 268|  0|  0|  0|
|         10|3146|2439|5097|5815|6058| 374| 50|  0|  0|
|         11|1764|1242|4309|5590|6983|1767|471|  0|  0|
|         12|2889|1573|4777|6512|6490| 363|  0|  0|  0|
+-----------+----+----+----+----+----+----+---+---+---+



### User Defined Functions (UDF)

**UDF**s allow for computations of values while looking at every input row in the DataFrame. They allow you to make your own function and import functionality from other **Python** libraries.

**Niezalecane użycie** Ale jak trzeba... to trzeba...

Funkcje **UDF** pozwalają na przetwarzanie wartości biorąc pod uwagę każdy  wiersz wejściowy w DataFrame. Pozwalają one tworzyć własne funkcje użytkownika i importować funkcje z innych bibliotek języka **Python**. Czy język Python służy do przetwarzania dużych zbiorów danych?

In [0]:
# Zdefiniuj funkcję przeliczania prędkości z mil na godzinę (mph) na kilometry na godzinę (km/h)

def mph_to_kmh(mph):
  kmh = float(mph * 1.60934)
  return kmh

# Zarejestruj tę funkcję jako UDF za pomocą `udf()`
mph_to_kmh = udf(mph_to_kmh, FloatType()) # Określono typ wyjściowy

In [0]:
# Usuń wartości null
nycflights_wind_visib_beaufort = \
(nycflights_wind_visib_beaufort
 .filter(nycflights_wind_visib_beaufort.
         wind_speed.
         isNotNull()))

In [74]:
nycflights_wind_visib_beaufort.select('wind_speed', mph_to_kmh('wind_speed').alias('wind_kmh')).show()

+----------+---------+
|wind_speed| wind_kmh|
+----------+---------+
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
|  10.35702|16.667967|
+----------+---------+
only showing top 20 rows



In [75]:
# Utwórz dwie nowe kolumny, które są konwersjami 
# prędkości wiatru z mph na km/h
(
  nycflights_wind_visib_beaufort
  .withColumn('wind_speed_kmh', mph_to_kmh('wind_speed'))
  .withColumn('wind_gust_kmh', mph_to_kmh('wind_gust'))
  .withColumnRenamed('wind_speed', 'wind_speed_mph')
  .withColumnRenamed('wind_gust', 'wind_gust_mph')
  .show()
)

+-------------------+-------+------+------+----+--------+--------------+-------------+-----+-----+-------------+-----+--------------+-------------+
|          timestamp|carrier|flight|origin|dest|wind_dir|wind_speed_mph|wind_gust_mph|visib|force|         name|month|wind_speed_kmh|wind_gust_kmh|
+-------------------+-------+------+------+----+--------+--------------+-------------+-----+-----+-------------+-----+--------------+-------------+
|2013-01-01 06:00:00|     B6|   371|   LGA| FLL|     270|      10.35702|    11.918652| 10.0|    3|gentle breeze|    1|     16.667967|    19.181162|
|2013-01-01 06:00:00|     MQ|  4650|   LGA| ATL|     270|      10.35702|    11.918652| 10.0|    3|gentle breeze|    1|     16.667967|    19.181162|
|2013-01-01 06:01:00|     B6|   343|   EWR| PBI|     270|      10.35702|    11.918652| 10.0|    3|gentle breeze|    1|     16.667967|    19.181162|
|2013-01-01 06:02:00|     DL|  1919|   LGA| MSP|     270|      10.35702|    11.918652| 10.0|    3|gentle breeze|

### Zapisywanie DataFrames do plików

Możemy łatwo zapisać dane DataFrame w różnych formatach plików.

In [0]:
# Zapisywanie do pliku CSV jest podobne do odczytu z pliku CSV
(crosstab_month_force
 .write
 .mode('overwrite')
 .format('com.databricks.spark.csv')
 .save('/Content/colab-examples/spark/data/save/nycflights13/crosstab_month_force/'))

In [0]:
# Jednak to zapisywanie do formatu plików Parquet jest zalecane ze względu na późniejszą wydajniejszą
# pracę z tego typu plikami
(crosstab_month_force
 .write
 .mode('overwrite')
 .parquet('/Content/colab-examples/spark/data/nycflights13/crosstab_month_force_parquet/'))